In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import datasets, transforms, models
from torch.autograd import Variable

# load data

In [2]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])


train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('./data', train=True, download=False,
                   transform=transforms.Compose([
                       transforms.RandomResizedCrop(max((224, 224))),
                       transforms.RandomHorizontalFlip(),
                       transforms.ToTensor(),
                       normalize
                   ])),
    batch_size=65, num_workers=2, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('./data', train=False,
                   transform=transforms.Compose([
                       transforms.Resize(int(max((224, 224))/224*256)),
                       transforms.CenterCrop(max((224, 224))),
                       transforms.ToTensor(),
                       normalize
                   ])),
    batch_size=65, num_workers=2, shuffle=True)

# train

In [3]:
class IdentityLayers(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, shortcut=None):
        super(IdentityLayers, self).__init__()
        self.conv1 = nn.Sequential(nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False),
                                   nn.BatchNorm2d(out_channels),
                                   nn.ReLU(inplace=True))
        
        self.conv2 = nn.Sequential(nn.Conv2d(out_channels, out_channels, kernel_size=3,
                                             stride=stride, padding=1, bias=False),
                                   nn.BatchNorm2d(out_channels),
                                   nn.ReLU(inplace=True))
        
        self.conv3 = nn.Sequential(nn.Conv2d(out_channels, out_channels*4, kernel_size=1, bias=False),
                                   nn.BatchNorm2d(out_channels * 4))

        self.shortcut = shortcut
        
        
    def forward(self, x):
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.conv3(out)

        if self.shortcut is not None:
            out += self.shortcut(x)
        else:
            out += x

        return F.relu(out, inplace=True)

In [4]:
class ResNet(nn.Module):
    def __init__(self):
        super(ResNet, self).__init__()
        self.conv = nn.Conv2d(3, 64, kernel_size=7,
                              stride=2, padding=3, bias=False)
        self.bn = nn.BatchNorm2d(64)

        self.in_channels = 64
        self.residual1 = self._make_indentity_block(64, 3)
        self.residual2 = self._make_indentity_block(128, 4, stride=2)
        self.residual3 = self._make_indentity_block(256, 6, stride=2)
        self.residual4 = self._make_indentity_block(512, 3, stride=2)

        self.fc = nn.Linear(512 * 4, 1000)

    def _make_indentity_block(self, out_channels, num_layers, stride=1):
        shortcut = None
        layers = []

        if stride != 1 or self.in_channels != out_channels * 4:
            shortcut = nn.Sequential(nn.Conv2d(self.in_channels, out_channels * 4,
                                               kernel_size=1, stride=stride, bias=False),
                                     nn.BatchNorm2d(out_channels * 4))

        layers.append(IdentityLayers(self.in_channels, out_channels,
                                     stride=stride, shortcut=shortcut))
        self.in_channels = out_channels * 4

        for i in range(1, num_layers):
            layers.append(IdentityLayers(self.in_channels, out_channels))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = F.relu(x, inplace=True)
        x = F.max_pool2d(x, kernel_size=3, stride=2, padding=1)

        x = self.residual1(x)
        x = self.residual2(x)
        x = self.residual3(x)
        x = self.residual4(x)

        x = F.avg_pool2d(x, kernel_size=7, stride=1)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [5]:
model = ResNet().cuda()
model

ResNet(
  (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (residual1): Sequential(
    (0): IdentityLayers(
      (conv1): Sequential(
        (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
        (2): ReLU(inplace)
      )
      (conv2): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
        (2): ReLU(inplace)
      )
      (conv3): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
      )
      (shortcut): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
      )
    )


In [8]:
optimizer = optim.SGD(model.parameters(), lr=0.001)
loss_func = nn.CrossEntropyLoss()
loss_func.cuda()

def train(train_loader):
    model.train()
    train_loss = 0
    for batch_idx, (x, y) in enumerate(train_loader):
        x, y = x.cuda(), y.cuda()
        b_x = Variable(x)
        b_y = Variable(y)
        
        optimizer.zero_grad()
        outputs = model(b_x)
        
        loss = loss_func(outputs, b_y)
        train_loss += loss.data[0]

        loss.backward()
        optimizer.step()

        if batch_idx == 100:
            break

    train_loss = train_loss / len(train_loader)
    
    return train_loss

def valid(test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    for i, (x, y) in enumerate(test_loader):
        x, y = x.cuda(), y.cuda()
        b_x = Variable(x, volatile=True)
        b_y = Variable(y, volatile=True)
        
        outputs = model(b_x)
        loss = loss_func(outputs, b_y)
        test_loss += loss.data[0]
        
        pred = outputs.data.max(1, keepdim=True)[1]
        correct += pred.eq(b_y.data.view_as(pred)).cpu().sum()
        total += b_y.size(0)

        if i == 10:
            break

    val_loss = test_loss / len(test_loader)
    val_acc = correct / total
    
    return val_loss, val_acc

In [9]:
epochs = 1
loss_list = []
val_loss_list = []
val_acc_list = []
for epoch in range(epochs):
    loss = train(train_loader)
    val_loss, val_acc = valid(test_loader)

    print('epoch %d, loss: %.4f val_loss: %.4f val_acc: %.4f'
          % (epoch, loss, val_loss, val_acc))
    
    # logging
    loss_list.append(loss)
    val_loss_list.append(val_loss)
    val_acc_list.append(val_acc)

epoch 0, loss: 0.5362 val_loss: 0.1905 val_acc: 0.1287
